In [ ]:

# CodeP3.4F25 - V.P. Carey, ME249 F25

#import useful packages
import keras
import pandas as pd
from keras.models import Sequential
import numpy as np
import keras.backend as kb
import tensorflow as tf
#the following 2 lines are only needed for Mac OS machines
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#create input data array

#convert to:
# BC's for PINN conduction model with HE enforcement in interior points

# x(cm), y(cm)   These are (x,y) coordinates of boundary location where temperature is specified from measurements
xdata = []
ND = 32
xdata =  [  [ 0.0, 0.0 ], [ 0.0, 0.5 ], [ 0.0, 1.0 ], [ 0.0, 1.5 ],[ 0.0, 2.0 ], [ 0.0, 2.5 ], [ 0.0, 3.0 ],
            [ 0.0, 3.5 ], [ 0.0, 4.0 ], [ 0.0, 4.5 ], [ 0.0, 5.0 ],
            [ 3.0, 0.0 ], [ 3.0, 0.5 ], [ 3.0, 1.0 ], [ 3.0, 1.5 ],[ 3.0, 2.0 ], [ 3.0, 2.5 ], [ 3.0, 3.0 ],
            [ 3.0, 3.5 ], [ 3.0, 4.0 ], [ 3.0, 4.5 ], [ 3.0, 5.0 ]]
   
xdata.append([  0.5, 0.0 ])
xdata.append([  1.0, 0.0 ])
xdata.append([  1.5, 0.0 ])
xdata.append([  2.0, 0.0 ])
xdata.append([  2.5, 0.0 ])

xdata.append([  0.5, 5.0 ])
xdata.append([  1.0, 5.0 ])
xdata.append([  1.5, 5.0 ])
xdata.append([  2.0, 5.0 ])
xdata.append([  2.5, 5.0 ])
  
    
# add more here if needed
   


xarray= np.array(xdata)
print (xdata)
print (xarray)


    
#convert to:
# meadian values of output variables

# T (deg C)   These are specified (measured) temperatures for each pair of (x,y) above

ydata = []

ydata = [ [ 50. ], [ 50. ], [ 50. ], [ 50. ], [ 50. ], [ 45. ], [ 40. ], [ 35. ], [ 30. ], [ 25. ], [ 20. ], 
         [ 40. ], [ 35. ],  [ 30. ], [ 25. ], [ 20. ], [ 15. ], [ 10. ], [ 10. ], [ 10. ], [ 10. ], [ 10. ] ]
    
ydata.append([ 50. ])
ydata.append([ 50. ])
ydata.append([ 50. ])
ydata.append([ 50. ])
ydata.append([ 45. ])

ydata.append([ 15. ])
ydata.append([ 10. ])
ydata.append([ 10. ])
ydata.append([ 10. ])
ydata.append([ 10. ])

# add more here if needed

yarray= np.array(ydata)
print (ydata)
print (yarray)

data_inputs = np.array(xdata)
data_outputs = np.array(ydata)

#boundary_x = ...  # x-coordinates for the boundary data
#boundary_y = ...  # y-coordinates for the boundary data
#boundary_T = ...  # Boundary temperature data (T_data)
    
# Copy the first column to a 1D numpy array
#vector = arr[:, 0].copy()
boundary_xnp = xarray[:, 0].copy()
# Copy the first column to a 1D numpy array
boundary_ynp = xarray[:, 1].copy()
boundary_Tnp = yarray[:, 0].copy()
    
# Convert the arrays to a TensorFlow tensors
boundary_x = tf.convert_to_tensor(boundary_xnp, dtype=tf.float32)
boundary_y = tf.convert_to_tensor(boundary_ynp, dtype=tf.float32)
boundary_T_ytrain = tf.convert_to_tensor(boundary_Tnp, dtype=tf.float32)

boundary_points_xtrain = tf.stack([boundary_x, boundary_y], axis=1)

print(boundary_points_xtrain)
print(boundary_T_ytrain)
    

In [ ]:
# define neural network model
    
    #As seen below, we have created four dense layers. 
    #A dense layer is a layer in neural network that’s fully connected. 
    #In other words, all the neurons in one layer are connected to all other neurons in the next layer.
    #In the first layer, we need to provide the input shape, which is 1 in our case. 
    #The activation function we have chosen is elu, which stands for exponential linear unit. 
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
    
# Create your model (for example, a simple fully connected feedforward NN)
# Assume 2D input (x, y),  Output temperature T
model = Sequential([
        Dense(5, input_dim=2, activation='elu'),  
        Dense(10, activation='elu'),
        Dense(1)  
    ])  
    
# Set the kappa value (this can be adjusted as per your needs)
kappa = 1.3
    
# Set the x_max and y_max values of your domain ( can adjust these according to your problem)
x_max = 3.0
y_max = 5.0

#print summary of model features
model.summary()

In [ ]:
# ADD CODE HERE for custom_loss_function <<========
    
    
from tensorflow.keras.callbacks import Callback
    
class EpochTracker(Callback):
    def __init__(self, N_epInit):
        super().__init__()
        self.N_epInit = N_epInit
        self.epochs_trained = 0
        
    def on_epoch_end(self, epoch, logs=None):
        self.epochs_trained = epoch
    
# Set the loss function with the required arguments
epoch_tracker = EpochTracker(N_epInit=2)
custom_loss = custom_loss_function(kappa=1.3, x_max=3.0, y_max=5.0, epoch_tracker=epoch_tracker)


    
# Compile the model with the custom loss
model.compile(optimizer='adam', loss=custom_loss)

    
    

In [ ]:
#### After the compilation of the model, we’ll try the fit method with 100 epochs.

    
#The fit method takes three parameters; namely, x, y, and number of epochs. 
#During model training, if all the batches of data are seen by the model once, 
#we say that one epoch has been completed.

 
##### RUN FIT 

# Assuming `boundary_points` and `boundary_T` are TensorFlow tensors

# Now, fit the model and pass the callback to `fit()`
historyData = model.fit(boundary_points_xtrain, boundary_T_ytrain, 
                epochs=100, batch_size=32, 
                callbacks=[epoch_tracker])


loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1
print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))

predictions = model.predict(data_inputs)


In [ ]:

predictions = model.predict(data_inputs)



#SETTING UP PLOT
%matplotlib inline
# importing the required module
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [8, 8] # for square canvas 
#========

'''CALCULATE PREDICTED VALUES AND RETRIEVE DATA VALUES TO PLOT'''

plt.scatter(predictions[:, 0], data_outputs[:, 0])
#plt.title('Genetic Algorithm training of Neural Network ==> output = alpha')
plt.xlabel('predicted output for NN (units)') 
plt.ylabel('data output (units)')
plt.loglog()
plt.xlim(xmax = 100, xmin = 1.)
plt.ylim(ymax = 100, ymin = 1.)
# Generate red y=x line
x_data = np.linspace(1.0, 100.0, num=3)
y_data = x_data
plt.plot(x_data, y_data, color='red')
plt.show()


 

In [ ]:
test = []
outpt=[]
predictions = model.predict(data_inputs)

#first point (row [0])comparison of data and prediction
# put in a loop to print comparion for all data points

xa=[]
ya=[]
za=[]

xao = np.linspace(0, 3, 7)  # 1D array for x values
yao = np.linspace(0, 5, 11)    # 1D array for y values
xa = np.linspace(0, 10, 77)    # 1D array for x values
ya = np.linspace(0, 10, 77)    # 1D array for y values
za = np.linspace(0, 10, 77)    # 1D array for z values

x =-0.50
ia=-1
while (x < 2.51):
    x = x + 0.5
    y = -0.5
    while (y < 4.51):
        ia=ia+1
        y = y + 0.50
        xa[ia]=x
        ya[ia]=y
       
        test = [[ x , y ]]
        testarray = np.array(test)
        outpt = model.predict(testarray)
        za[ia] = outpt
        print ('x= ', testarray[0][0], ', y= ', testarray[0][1],'T= ', outpt[0][0])
        


X, Y, Z = xa, ya, za

import sys
#import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Plot X,Y,Z

# Creating figure
#fig = plt.figure(figsize =(16, 9))  
#ax = plt.axes(projection ='3d') 

fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection ='3d')
#ax.plot_trisurf(X, Y, Z, color='white', edgecolors='grey', alpha=0.5)
ax.plot_trisurf(X, Y, Z,
                linewidth = 0.2,
                antialiased = True
                , cmap='viridis');
ax.invert_yaxis()
ax.invert_xaxis()
plt.show()



# Plot the data
plt.figure(figsize=(5, 6))
plt.tripcolor(xa, ya, za, shading='flat', cmap='viridis')
plt.colorbar(label='Z value')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Heat Map of Z values over X and Y domain')
plt.show()
